## Web Scraping con bs4 (BeautifulSoup)

**`BeautifulSoup`** es una biblioteca de python para extraer contenido de ficheros **`HTML`** y **`XML`**.

```sh
pip install beautifulsoup4
```

Primero usamos **`requests`** para tener acceso al contenido **`HTML`** de las páginas web, luego usamos **`BeautifulSoup`** para extraer la información de ese conteido.

In [1]:
pip install beautifulsoup4

In [2]:
import numpy as np

import requests
import bs4 # Para ver la versión
from bs4 import BeautifulSoup

In [3]:
# Versiones

print(f"numpy=={np.__version__}")
print(f"requests=={requests.__version__}")
print(f"bs4=={bs4.__version__}")

numpy==2.2.0
requests==2.32.3
bs4==4.12.3


### requests

**`requests.get()`** toma un **`url`** (o enlace) y retorna la **"respuesta"** del servidor. Este nuevo objeto también extrae el código **`HTML`** del **`url`**.

Cada **"request"** toma un tiempo de respuesta, por lo que si intentamos hacer muchos **"request"** en un plazo corto de tiempo nuestro I.P. será baneado de la página web para evitar que colapse por el gran número de **"requests"**. En el peor de los casos la página recibirá tantos **"requests"** que colapsará y la dejaremos fuera de servicio.

El **`CAPTCHA`** es una herramienta que utilizan las páginas web para evitar el **web scraping** o evitar que un mismo usuario usando bots colapse la página.

_**"Completely Automated Public Turing test to tell Computers and Humans Apart"**_

In [ ]:
url = "https://google.com/"

response = requests.get(url)

print(response)

print(bool(response))

In [ ]:
# El atributo .text retorna el HTML de la página 

print(response.text)

In [ ]:
type(response.text)

### BeautifulSoup

Si la respuesta del **`requests`** es positiva podemos pasar este objeto **`requests.get()`** a **`BeautifulSoup`** para que nos ayude a filtrar la información y extraerla más fácil. 

Los métodos más comunes de **`BeautifulSoup`** son:

|Método           |Descripción                                                                                        |
|-----------------|---------------------------------------------------------------------------------------------------|
|**`.body`**      | Retorna el contenido dentro de la etiqueta **`body`**.                                            |
|**`.title`**     | Retorna el titulo del **HTML**.                                                                   |
|**`.find()`**    | Busca en el **HTML** y retorna la primera ocurrencia del filtro en un objeto **`bs4`**.           |
|**`.find_all()`**| Busca en el **HTML** y retorna todas las ocurrencias del filtro en una lista de objetos **`bs4`**.|
|**`.text`**      | Retorna el contenido de un objeto **`bs4`** en un **`str`**.                                          |

In [ ]:
soup = BeautifulSoup(response.text, "html.parser")

In [ ]:
soup

In [ ]:
type(soup)

In [ ]:
# sensacine

url = "https://www.sensacine.com/peliculas/en-cartelera/cines/"

response = requests.get(url)
soup = BeautifulSoup(response.text, "html.parser")

In [ ]:
response.status_code

In [ ]:
soup.body

In [ ]:
# Esto retorna un objeto bs4
soup.title

In [ ]:
# Con .text lo convertimos a str

soup.title.text

In [ ]:
type(soup.title.text)

### Filtros con bs4

In [ ]:
# Encontrar la primera etiqueta h2

soup.find("h2").text

In [ ]:
# Con .find_all() encontramos todas las etiquetas y nos retorna una lista con objetos bs4

soup.find_all("h2")

In [ ]:
# Encuentra la primera etiqueta a

soup.find("a")

In [ ]:
# Encuentra todas las etiquetas a

soup.find_all("a")

In [ ]:
len(soup.find_all("a"))

In [ ]:
# Podemos encadenar filtros siempre que nos retorne un objeto bs4

soup.find("h2")

In [ ]:
soup.find("h2").find("a")

In [ ]:
soup.find("h2").find("a")["href"]

In [ ]:
soup.find("h2").find("a").text

In [ ]:
soup.find("h2").find("a")["class"]

Cuando veamos una etiqueta con **class** en ella podemos usarla para hacer un filtro más específico.
```html
<li class="mdl">
```

Como en Python **`class`** es una palabra reservada, debemos usar **`class_`**.

In [ ]:
soup.find("li", class_ = "mdl")

In [ ]:
soup.find_all("li", class_ = "mdl")

In [ ]:
soup.find("span", class_ = "stareval-note")

In [ ]:
soup.find_all("span", class_ = "stareval-note")

Otra forma de usar estos filtros de **`bs4`** es atraves del parametro **`attrs`** de un diccionario.

Este diccionario tendrá como llave el nombre de las etiquetas y como valor el valor asociado a esa etiqueta:

In [ ]:
dicc_bs4 = {"class" : "stareval-note"}

soup.find("span", attrs = dicc_bs4)

In [ ]:
soup.find("span", attrs = dicc_bs4).text

Utilizando bucles podemos iterar sobre los **`.find_all()`** y sacar información de forma automatizada.

In [ ]:
# En la etiqueta h3 tenemos el url y el titulo de la pelicula

soup.find_all("h2")

In [ ]:
soup.find("h2").find("a").text

In [ ]:
titulos = list()

urls = list()

for bs in soup.find_all("h2"):
    
    try:
        titulo = bs.find("a").text
    except:
        titulo = np.nan
        
    try:
        url = bs.find("a")["href"]
    except:
        url = np.nan
    
    titulos.append(titulo)
    urls.append(url)
    
    
for t, u in zip(titulos, urls):
    print(t, u)

Toda esta información se puede agrupar en un DataFrame y convertir a **`.csv`**

In [ ]:
import pandas as pd

In [ ]:
df = pd.DataFrame()

df["titulo"] = titulos

df["urls"] = urls

In [ ]:
df

In [ ]:
df["urls"] = df["urls"].apply(lambda x : f"https://www.sensacine.com{x}" if type(x) == str else x)

In [ ]:
df

In [ ]:
# path_or_buf = Nombre del archivo terminado en .csv
# index = False para que no se guarde el indice como nueva columna en el csv
# sep = "," para que el separador sea ",", se puede cambiar por cualquier otro separador.

df.to_csv(path_or_buf = "sensacine.csv", index = False, sep = ",")

In [ ]:
df.to_pickle(path = "sensacine.pkl")

### Agregar al DataFrame la duración de cada pelicula y su puntaje:

In [ ]:
soup.find("div", class_ = "meta-body-item meta-body-info").text.replace("\n", "").split("/")#[1]

In [ ]:
# [np.nan if len(x.text.replace("\n", "").split("/")) < 3 else x.text.replace("\n", "").split("/")[1] for x in soup.find_all("div", class_ = "meta-body-item meta-body-info")]

In [ ]:
lista_duraciones = list()

for x in soup.find_all("div", class_ = "meta-body-item meta-body-info"):
    
    x = x.text.replace("\n", "").split("/")
    
    if len(x) < 3:
        
        lista_duraciones.append(np.nan)
        
    else:
        
        lista_duraciones.append(x[1])
        
lista_duraciones

In [ ]:
soup.find_all("li", class_ = "mdl")[0]

In [ ]:
soup.find_all("li", class_ = "mdl")[1]

In [ ]:
len(soup.find_all("li", class_ = "mdl"))

In [ ]:
lista_puntaciones = list()

for pelicula in soup.find_all("li", class_ = "mdl"):
    
    if pelicula.find("a") != None:
    
        try:
            x = float(pelicula.find("div", class_ = "rating-item").find("span", class_ = "stareval-note").text.replace(",", "."))

        except:

            x = np.nan
    
        lista_puntaciones.append(x)
        
lista_puntaciones

In [ ]:
df["duracion"] = lista_duraciones

df["puntuacion"] = lista_puntaciones

df

In [ ]:
df.info()

### Información de 1 pelicula:

Vamos a entrar al primer **`url`** y sacar toda su información.

In [ ]:
url = "https://www.sensacine.com/peliculas/pelicula-246641/"

response = requests.get(url)
soup = BeautifulSoup(response.text, "html.parser")

soup

In [ ]:
# Titulo

soup.find("div", class_ = "titlebar titlebar-page").text.strip()

In [ ]:
# Fecha

dict_bs4 = {"class" : "meta-body-item meta-body-info"}

soup.find("div", attrs = dict_bs4).text.strip().split("\n")[0]

In [ ]:
# Duración

dict_bs4 = {"class" : "meta-body-item meta-body-info"}

soup.find("div", attrs = dict_bs4).text.replace("\n", "").split("/")[1]

In [ ]:
# Guion 

soup.find_all("div", class_ = "meta-body-item meta-body-direction")[1].find_all("span")[1].text

In [ ]:
# Reparto 

soup.find("div", class_ = "meta-body-item meta-body-actor").find_all("span")[1:]

In [ ]:
[x.text for x in soup.find("div", class_ = "meta-body-item meta-body-actor").find_all("span")[1:]]

In [ ]:
# Géneros

dict_bs4 = {"class" : "meta-body-item meta-body-info"}

soup.find("div", attrs = dict_bs4).text.replace("\n", "").split("/")[-1].split(",")

In [ ]:
# Sinopsis

soup.find("section", attrs = {"class" : "section ovw ovw-synopsis"}).find("div", class_ = "content-txt").text

In [ ]:
from time import sleep

In [ ]:
df["urls"]

In [ ]:
# Crear un DataFrame

titulos = list()
fechas = list()
duraciones = list()
guiones = list()
repartos = list()
generos = list()
sinopsises = list()

for url in df["urls"]:
    
    print(url)
    
    try:
        response = requests.get(url)
        soup = BeautifulSoup(response.text, "html.parser")

        titulo = soup.find("div", class_ = "titlebar titlebar-page").text
        fecha = soup.find("div", attrs = dict_bs4).text.strip().split("\n")[0]
        duracion = soup.find("div", class_ = "meta-body-item meta-body-info").text.replace("\n", "").split("/")[1]
        guion = soup.find_all("div", class_ = "meta-body-item meta-body-direction")[1].find_all("span")[1].text
        reparto = ", ".join([x.text for x in soup.find("div", class_ = "meta-body-item meta-body-actor").find_all("span")[1:]])
        genero = " ".join(soup.find("div", class_ = "meta-body-item meta-body-info").text.replace("\n", "").split("/")[-1].split(","))
        sinopsis = soup.find("section", class_ = "section ovw ovw-synopsis").find("div", class_ = "content-txt").text

        titulos.append(titulo)
        fechas.append(fecha)
        duraciones.append(duracion)
        guiones.append(guion)
        repartos.append(reparto)
        generos.append(genero)
        sinopsises.append(sinopsis)
        
    except:
        
        titulos.append(np.nan)
        fechas.append(np.nan)
        duraciones.append(np.nan)
        guiones.append(np.nan)
        repartos.append(np.nan)
        generos.append(np.nan)
        sinopsises.append(np.nan)
    
    sleep(3)

Muchas veces sucede que no todas las páginas tienen la misma información, por lo que tenemos que asegurarnos de que si no existe un dato lo sustituimos con un np.nan. Por eso podemos usar:
```python
try:
    
except:
```

In [ ]:
# Crear un DataFrame

titulos = list()
fechas = list()
duraciones = list()
guiones = list()
repartos = list()
generos = list()
sinopsises = list()

for url in df["urls"]:
    
    print(url)
    
    if type(url) == str:
        response = requests.get(url)
        soup = BeautifulSoup(response.text, "html.parser")
    
#### Titulo #######################################################################################################   

        try:
            titulo = soup.find("div", class_ = "titlebar titlebar-page").text

        except:
            titulo = np.nan
        
#### Fecha ########################################################################################################   

        try:
            fecha = soup.find("div", class_ = "meta-body-item meta-body-info").text.strip().split("\n")[0]

        except:
            fecha = np.nan

#### Duracion #####################################################################################################

        try:
            duracion = soup.find("div", attrs = dict_bs4).text.replace("\n", "").split("/")[1]

        except:
            duracion = np.nan

#### Guion ########################################################################################################

        try:
            guion = soup.find_all("div", class_ = "meta-body-item meta-body-direction")[1].find_all("span")[1].text

        except:
            guion = np.nan

#### Reparto ######################################################################################################

        try:
            reparto = ", ".join([x.text for x in soup.find("div", class_ = "meta-body-item meta-body-actor").find_all("span")[1:]])

        except:
            reparto = np.nan

#### Genero #######################################################################################################

        try:
            genero = " ".join(soup.find("div", attrs = dict_bs4).text.replace("\n", "").split("/")[-1].split(","))

        except:
            genero = np.nan

#### Sinopsis #####################################################################################################

        try:
            sinopsis = soup.find("section", attrs = {"class" : "section ovw ovw-synopsis"}).find("div", class_ = "content-txt").text

        except:
            sinopsis = np.nan
        
##################################################################################################################

        # appends
    
        titulos.append(titulo)
        fechas.append(fecha)
        duraciones.append(duracion)
        guiones.append(guion)
        repartos.append(reparto)
        generos.append(genero)
        sinopsises.append(sinopsis)

        sleep(3)

In [ ]:
df_peliculas = pd.DataFrame()

df_peliculas["titulo"] = titulos
df_peliculas["fecha"] = fechas
df_peliculas["duracion"] = duraciones
df_peliculas["guion"] = guiones
df_peliculas["reparto"] = repartos
df_peliculas["genero"] = generos
df_peliculas["sinopsis"] = sinopsises

df_peliculas["url"] = df["urls"]

In [ ]:
df_peliculas

### Ejercicio: Modificar el código para que no hayan errores

In [ ]:
# Crear un DataFrame

titulos = list()
fechas = list()
duraciones = list()
guiones = list()
repartos = list()
generos = list()
sinopsises = list()

for url in df["urls"]:
    
    print(url)
    
    
    if type(url) == str:
        response = requests.get(url)
        soup = BeautifulSoup(response.text, "html.parser")
    
#### Titulo #######################################################################################################   

        try:
            titulo = soup.find("div", class_ = "titlebar titlebar-page").text

        except:
            titulo = np.nan
        
#### Fecha ########################################################################################################   

        try:
            fecha = soup.find("div", class_ = "meta-body-item meta-body-info").text.strip().split("\n")[0]

        except:
            fecha = np.nan

#### Duracion #####################################################################################################

        try:
            duracion = soup.find("div", attrs = dict_bs4).text.replace("\n", "").split("/")

            if len(duracion) < 3:

                duracion = np.nan

            else:

                duracion = duracion[1]

        except:
            duracion = np.nan

#### Guion ########################################################################################################

        try:
            guion = soup.find_all("div", class_ = "meta-body-item meta-body-direction")[1].find_all("span")[1].text

        except:
            guion = np.nan

#### Reparto ######################################################################################################

        try:
            reparto = ", ".join([x.text for x in soup.find("div", class_ = "meta-body-item meta-body-actor").find_all("span")[1:]])

        except:
            reparto = np.nan

#### Genero #######################################################################################################

        try:
            genero = " ".join(soup.find("div", attrs = dict_bs4).text.replace("\n", "").split("/")[-1].split(","))

        except:
            genero = np.nan

#### Sinopsis #####################################################################################################

        try:
            sinopsis = soup.find("section", attrs = {"class" : "section ovw ovw-synopsis"}).find("div", class_ = "content-txt").text

        except:
            sinopsis = np.nan
        
##################################################################################################################

        # appends

        titulos.append(titulo)
        fechas.append(fecha)
        duraciones.append(duracion)
        guiones.append(guion)
        repartos.append(reparto)
        generos.append(genero)
        sinopsises.append(sinopsis)

        sleep(3)

In [ ]:
df_peliculas = pd.DataFrame()

df_peliculas["titulo"] = titulos
df_peliculas["fecha"] = fechas
df_peliculas["duracion"] = duraciones
df_peliculas["guion"] = guiones
df_peliculas["reparto"] = repartos
df_peliculas["genero"] = generos
df_peliculas["sinopsis"] = sinopsises

df_peliculas["url"] = df["urls"]
df_peliculas["puntuacion"] = df["puntuacion"]

df_peliculas

In [ ]:
df_peliculas.to_csv("info_peliculas.csv", index = False, sep = ",")

In [ ]:
################################################################################################################################